In [ ]:
import ixmp
import pyam
import nomenclature

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")
eu_climate_rp = nomenclature.RegionProcessor.from_directory("../mappings")

In [ ]:
source_db = ixmp.Platform("openentrance-internal")

In [ ]:
oe_db = ixmp.Platform("openentrance")

In [ ]:
props = source_db.scenario_list()

In [ ]:
props.model.unique()

In [ ]:
model = "GENeSYS-MOD 3.1"

In [ ]:
list()

In [ ]:
scenarios = props[props.model == model].scenario

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=source_db, model=model, scenario=s).timeseries()
    )
    for s in scenarios
]
df = pyam.concat(scenario_list)

In [ ]:
dict()

In [ ]:
df.rename(region={"Slovak Republic": "Slovakia"}, inplace=True)

In [ ]:
# migration to public openENTRANCE database
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(oe_db, _model, _scenario, version="new")
    scen.add_timeseries(_df.data)
    scen.commit(f"Migrated from H2020 openENTRANCE (project-internal) database")
    scen.set_as_default()

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
df.rename(unit={
    "USD_2010/kWr": "US$2010/kW",
    "Gpkm/yr": "billion pkm/yr",
    "Gtkm/yr": "billion tkm/yr",
}, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(_df.data)
    scen.commit(f"Migrated from H2020 openENTRANCE (project-internal) database")
    scen.set_as_default()

    scen.set_meta({"Source": "H2020 openENTRANCE"})